In [1]:
import argparse
import roypy
import time
import queue
from sample_camera_info import print_camera_info
from roypy_sample_utils import CameraOpener, add_camera_opener_options
#from roypy_platform_utils import PlatformHelper

import numpy as np
import matplotlib.pyplot as plt
import cv2
import random
import math
import socket_sender


try:
    import roypycy
except ImportError:
    print("Pico Flexx backend requirements (roypycy) not installed properly")
    raise

In [2]:
# # RANSAM

def Dis_pt2plane(pts, a, b, c, d):
    """
    Compute the distance from points to the plane
    """
    normal = math.sqrt(a*a+b*b+c*c)
    if normal == 0:
        normal = 1
    
    v = np.array([a,b,c])
    dis = abs(np.dot(pts,v.T)+d)/normal
    return dis

def get_Plane(sampts):
    """
    Compute the equation of the plane
    """
    p1 = sampts[0]
    p2 = sampts[1]
    p3 = sampts[2]
    
    a = ( (p2[1]-p1[1])*(p3[2]-p1[2])-(p2[2]-p1[2])*(p3[1]-p1[1]) )
    b = ( (p2[2]-p1[2])*(p3[0]-p1[0])-(p2[0]-p1[0])*(p3[2]-p1[2]) )
    c = ( (p2[0]-p1[0])*(p3[1]-p1[1])-(p2[1]-p1[1])*(p3[0]-p1[0]) )
    d = ( 0-(a*p1[0]+b*p1[1]+c*p1[2]) )
    
    return a,b,c,d

# def Random3points(points3D, ConfidenceIndex):
#     """
#     Random choose 3 Confidence points
#     """
#     sample_number = 3
#     sample_point_index = random.sample(range(ConfidenceIndex.shape[0]), sample_number)
#     sample_points = np.zeros((sample_number,3))
#     for i in range(sample_number):
#         Confidence_point_index = sample_point_index[i]
#         index = ConfidenceIndex[Confidence_point_index]
#         y = index // points3D.shape[1]
#         x = index % points3D.shape[1]
#         sample_points[i] = points3D[y][x]
#     return sample_points

def Random3points(points3D):
    sample_number = 30
    sample_point_index = random.sample(range(points3D.shape[0]*points3D.shape[1]), sample_number)
    sample_points = np.zeros((3,3))
    num = 0
    for i in range(sample_number):
        index = sample_point_index[i]
        y = index // points3D.shape[1]
        x = index % points3D.shape[1]
        
        point = points3D[y][x]
        if(point[0] != 0 or point[1] != 0 or point[2] != 0):
            sample_points[num] = points3D[y][x]
            num = num + 1
        
        if(num == 3):
            break
    return sample_points

def get_inliner_num(points3D,a,b,c,d,inliner_threshold,Confidence_img):
    """
    Compute the liner points which distance to plane < threshold
    Also get distance from points to the plane (new Depth Image which re-project depth pixels in surface plane)
    """
    inliner_num = 0
    
    dist = Dis_pt2plane(points3D,a,b,c,d)
    inliner_mask = dist < inliner_threshold
    inliner_mask = np.logical_and(inliner_mask, Confidence_img)
    inliner_num = np.sum(inliner_mask)
    return inliner_num, inliner_mask, dist

def RANSAM(points3D, Confidence_img, ransac_iteration = 1000, inliner_threshold = 0.01):
    best_inlinernum = -1
    best_inlinernum = 0
    best_plane = np.zeros((1,4))
    best_depthImage = np.zeros((points3D.shape[0],points3D.shape[1]))
    best_plane_mask = np.zeros((points3D.shape[0],points3D.shape[1]))
    best_sampts = np.zeros((3,3))
    
#     print(points3D.shape,points3D[80:90,110])
    for i in range(ransac_iteration):
        sampts = Random3points(points3D)
        a,b,c,d = get_Plane(sampts)
        
        inliner_num, inliner_mask, depthImage = get_inliner_num(points3D,a,b,c,d,inliner_threshold,Confidence_img)
        if(inliner_num > best_inlinernum):
            best_inlinernum = inliner_num
            best_plane = np.array([a,b,c,d])
            best_plane_mask = inliner_mask
            best_depthImage = depthImage
            best_sampts = sampts
            
    #print("Inliner Number\n", best_inlinernum)
    #print("Inliner plane\n", best_plane)
    return best_plane, best_depthImage, best_plane_mask, best_sampts

In [46]:
def feet_detection(depthImg, Confidence_img, height = 0.05, feet_height = 0.1):
    highter_region = depthImg > height
    lower_region = depthImg < feet_height
    feet_region = np.logical_and(highter_region, lower_region)
    Confidence_feet_region = np.logical_and(feet_region, Confidence_img)
    
    return MorphologyEx(Confidence_feet_region.astype(np.uint8))

In [48]:
def MorphologyEx(img):
    kernel_size = 5 #7
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT,(kernel_size, kernel_size))
    #膨胀之后再腐蚀，在用来关闭前景对象里的小洞或小黑点
    #开运算用于移除由图像噪音形成的斑点
    opened = cv2.morphologyEx(img, cv2.MORPH_OPEN, kernel)
    closing = cv2.morphologyEx(img,cv2.MORPH_CLOSE,kernel)
    
    kernel_size = 3
    plane_blur = cv2.GaussianBlur(closing,(kernel_size, kernel_size), 0)
    return plane_blur

In [42]:


class MyListener(roypy.IDepthDataListener):
    def __init__(self, xqueue, yqueue, zqueue):
        super(MyListener, self).__init__()
        self.xqueue = xqueue
        self.yqueue = yqueue
        self.zqueue = zqueue
        self.Listening = True

    def onNewData(self, data):   
        if(self.Listening):
            t_time = time.time()
            
            xvalues = []
            yvalues = []
            zvalues = []
            
            values = roypycy.get_backend_data(data)

            xvalues = values.x
            yvalues = values.y
            zvalues = values.z

            xarray = np.asarray(xvalues)
            yarray = np.asarray(yvalues)
            zarray = np.asarray(zvalues)
            
            
            q_x = xarray.reshape (-1, data.width)        
            self.xqueue.put(q_x)
            q_y = yarray.reshape (-1, data.width)        
            self.yqueue.put(q_y)
            q_z = zarray.reshape (-1, data.width)        
            self.zqueue.put(q_z)
            
            #print('store time:', (time.time()-t_time))

    def paint (self, data, name):
        """Called in the main thread, with data containing one of the items that was added to the
        queue in onNewData.
        """
        cv2.namedWindow(name, cv2.WINDOW_NORMAL)
        cv2.imshow(name, data)
        cv2.waitKey(1)


def main ():
    parser = argparse.ArgumentParser (usage = __doc__)
    add_camera_opener_options (parser)
    parser.add_argument ("--seconds", type=int, default=15, help="duration to capture data")
    timer_show = False
    
    Replay = True
    if(Replay == True):
        options = parser.parse_args(args=['--rrf', 'feet02.rrf','--seconds', '25'])
    else:
        options = parser.parse_args(args=['--seconds', '30'])

    opener = CameraOpener (options)
    cam = opener.open_camera ()
    
    if(Replay == False):
        cam.setUseCase('MODE_5_35FPS_600')#MODE_9_5FPS_2000 MODE_5_45FPS_500

    #Print camera information
    print_camera_info (cam)
    print("isConnected", cam.isConnected())
    print("getFrameRate", cam.getFrameRate())
    print("UseCase",cam.getCurrentUseCase())

    # we will use this queue to synchronize the callback with the main
    # thread, as drawing should happen in the main thread 
    x = queue.LifoQueue()
    y = queue.LifoQueue()
    z = queue.LifoQueue()
    l = MyListener(x,y,z)
    cam.registerDataListener(l)
    cam.startCapture()
    
    # create a loop that will run for a time (default 15 seconds)
    process_event_queue (x, y, z, l, options.seconds, cam)
    cam.stopCapture()
    socket_sender.close_socket()
    
    cv2.destroyAllWindows()
    

def process_event_queue (x,y,z, painter, seconds, cam):
    
    # create a loop that will run for the given amount of time
    t_end = time.time() + seconds
    while time.time() < t_end:
        
#     key = ''
#     print("  Quit : Q\n")
#     while 1 :
        try:
            # try to retrieve an item from the queue.
            # this will block until an item can be retrieved
            # or the timeout of 1 second is hit
            t_time = time.time()
            
            item_x = x.get(True, 0.5)
            item_y = y.get(True, 0.5)
            item_z = z.get(True, 0.5)
            points3D = np.dstack((item_x,item_y,item_z))
            #print('queue time:', (time.time()-t_time))
        except queue.Empty:
            # this will be thrown when the timeout is hit
            break
        else:
            
            t_time = time.time()
            #if there have z value(z != 0) ==> True
            Confidence_img = points3D[:,:,2] != 0 
#             surface_plane, depthImg, plane_mask, best_sampts = RANSAM(points3D, ConfidenceIndex, ransac_iteration = 500, inliner_threshold = 0.003)
            surface_plane, depthImg, plane_mask, best_sampts = RANSAM(points3D, Confidence_img, ransac_iteration = 50, inliner_threshold = 0.01)#1cm  0.003
#             print('Ransam time:', (time.time()-t_time))
            
            #turn bool img to uint8
            plane_img = plane_mask.astype(np.uint8)*255 
            
            feet_img = feet_detection(depthImg, Confidence_img, height = 0.02, feet_height = 0.1)
           
                
            #show image
            painter.paint (depthImg, 'Depth')
            painter.paint (plane_img, 'plane')
            painter.paint (feet_img.astype(np.uint8)*255, 'feet_img')
            painter.paint (Confidence_img.astype(np.uint8)*255, 'Confidence_img')

                
        if(cv2.waitKey(10) & 0xFF == 113):
            break


In [49]:
main()

Filename: feet02.rrf
        Camera information
Type:            PICOFLEXX
Width:           224
Height:          171
Operation modes: 1
    MODE_PLAYBACK
        this operation mode has 3811767088 streams
Lens parameters: 9
    ('cx', 118.28559112548828)
    ('cy', 87.74105072021484)
    ('fx', 213.8031768798828)
    ('fy', 213.8031768798828)
    ('k1', 0.4155448377132416)
    ('k2', -4.7316107749938965)
    ('k3', 8.45906925201416)
    ('p1', 7.605663946304829e-16)
    ('p2', 4.939198934392371e-16)
CameraInfo items: 0
isConnected True
getFrameRate 0
UseCase MODE_PLAYBACK
close the socket successful
